In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from gensim.parsing.preprocessing import strip_non_alphanum, strip_punctuation
import torch
import torchtext
from sklearn.model_selection import *
from torch import nn
import gensim.downloader as api

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator
from torchtext.datasets import IMDB
from transformers import BertTokenizer
from transformers import BertModel
from skorch import NeuralNetClassifier
from skorch.callbacks import Freezer
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.pipeline import * 
from sklearn.compose import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.neural_network import *
from sklearn.preprocessing import *
from skorch.callbacks import ProgressBar
from util import *
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import fse
from fse.models import uSIF

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [3]:
subset = list((set(tier1)-set(["PersonalizedProduct"]))&set(all_tiers_100))
subset

['Manufacturing',
 'AnalysisAndModeling',
 'Imaging',
 'AnatomicalTarget',
 'SurgicalMethod',
 'SpecificationofUse']

In [51]:
nice_subset = [tier_translations[x]  for x in subset]
nice_subset

['Manufacturing',
 'Analysis and Modeling',
 'Imaging',
 'Anatomical Target',
 'Surgical Method',
 'Specification of Use']

In [52]:
prefixed_subset = [f"LABEL: {x}" for x in nice_subset]
prefixed_subset

['LABEL: Manufacturing',
 'LABEL: Analysis and Modeling',
 'LABEL: Imaging',
 'LABEL: Anatomical Target',
 'LABEL: Surgical Method',
 'LABEL: Specification of Use']

In [4]:
training_labels = training_set[subset]
testing_labels = testing_set[subset]

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("output/training-triplets-distilbert-base-uncased-2020-11-20_18-25-21/")

In [53]:
embedded_subset = model.encode(prefixed_subset)

In [9]:
training_embeddings = model.encode(training_set.abstract.values)

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
testing_embeddings = model.encode(testing_set.abstract.values)

In [61]:
labeling = cosine_similarity(testing_embeddings, embedded_subset)

In [59]:
testing_embeddings.shape

(243, 768)

In [60]:
embedded_subset.shape

(6, 768)

In [65]:
testing_labels.head()

,Manufacturing,AnalysisAndModeling,Imaging,AnatomicalTarget,SurgicalMethod,SpecificationofUse
0,True,True,False,True,False,False
1,False,True,True,True,False,False
2,False,False,False,True,True,False
3,False,False,True,False,False,True
4,True,True,True,True,False,False


In [63]:
labeling[0] 
# ['LABEL: Manufacturing',
# 'LABEL: Analysis and Modeling',
# 'LABEL: Imaging',
# 'LABEL: Anatomical Target',
# 'LABEL: Surgical Method',
 #'LABEL: Specification of Use']

array([-0.12024341,  0.15846373,  0.27344263, -0.05742005,  0.02295958,
        0.09729078], dtype=float32)

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3, metric='cosine', n_jobs=-1)

In [48]:
classifier.fit(training_embeddings, training_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [49]:
predictions = classifier.predict(testing_embeddings)

In [50]:
print(classification_report(testing_labels, predictions, target_names=subset))

                     precision    recall  f1-score   support

      Manufacturing       0.37      0.48      0.42        83
AnalysisAndModeling       0.36      0.45      0.40        84
            Imaging       0.58      0.62      0.60       133
   AnatomicalTarget       0.72      0.70      0.71       164
     SurgicalMethod       0.36      0.20      0.26        40
 SpecificationofUse       0.37      0.33      0.35        79

          micro avg       0.51      0.53      0.52       583
          macro avg       0.46      0.46      0.46       583
       weighted avg       0.51      0.53      0.52       583
        samples avg       0.50      0.54      0.48       583



/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
